In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


In [2]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: Udemy_ML
Azure region: eastus2
Subscription id: ef78944e-d555-4976-84de-e8954c8a9357
Resource group: Machine_Learning


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "capstone-CPU"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

# can poll for a minimum number of nodes and for a specific timeout. 
# if no min node count is provided it uses the scale settings for the cluster
compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=2)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-02-20T18:20:45.245000+00:00', 'errors': None, 'creationTime': '2021-01-19T17:25:38.254875+00:00', 'modifiedTime': '2021-01-19T17:25:54.517722+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [5]:
import os

project_folder = './sklearn-DM'
os.makedirs(project_folder, exist_ok=True)

In [9]:
import shutil

shutil.copy('train_dm.py', project_folder)

'./sklearn-DM/train_dm.py'

In [7]:
from azureml.core import Experiment

experiment_name = 'train_dm_hyperdrive'
experiment = Experiment(ws, name=experiment_name)



In [8]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn
- pip:
  - azureml-defaults

Writing conda_dependencies.yml


In [11]:
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

In [12]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=project_folder,
                      script='train_dm.py',
                      arguments=['--kernel', 'linear', 
                                    '--penalty', 1.0, 
                                    '--input-data', early_stage_diabetes_transformed.as_named_input('diabetes_dataset')]],
                      compute_target=compute_target,
                      environment=sklearn_env)



In [13]:
run = experiment.submit(src)

In [14]:
from azureml.widgets import RunDetails

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [15]:
run.wait_for_completion(show_output=True)


RunId: train_dm_hyperdrive_1613921318_6c834ed9
Web View: https://ml.azure.com/experiments/train_dm_hyperdrive/runs/train_dm_hyperdrive_1613921318_6c834ed9?wsid=/subscriptions/ef78944e-d555-4976-84de-e8954c8a9357/resourcegroups/Machine_Learning/workspaces/Udemy_ML

Streaming azureml-logs/20_image_build_log.txt

2021/02/21 15:28:47 Downloading source code...
2021/02/21 15:28:49 Finished downloading source code
2021/02/21 15:28:49 Creating Docker network: acb_default_network, driver: 'bridge'
2021/02/21 15:28:50 Successfully set up Docker network: acb_default_network
2021/02/21 15:28:50 Setting up Docker configuration...
2021/02/21 15:28:51 Successfully set up Docker configuration
2021/02/21 15:28:51 Logging in to registry: 1a6e7985b3de4689a1d267a81fc49464.azurecr.io
2021/02/21 15:28:52 Successfully logged into 1a6e7985b3de4689a1d267a81fc49464.azurecr.io
2021/02/21 15:28:52 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2021/02/21 

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "AzureMLCompute job failed.\nJobFailed: Submitted script failed with a non-zero exit code; see the driver log file for details.\n\tReason: Job failed with non-zero exit Code",
        "messageFormat": "{Message}",
        "messageParameters": {
            "Message": "AzureMLCompute job failed.\nJobFailed: Submitted script failed with a non-zero exit code; see the driver log file for details.\n\tReason: Job failed with non-zero exit Code"
        },
        "details": [],
        "innerError": {
            "code": "BadArgument",
            "innerError": {
                "code": "AmlComputeBadRequest"
            }
        }
    },
    "correlation": {
        "operation": null,
        "request": "97deb6786b812666"
    },
    "environment": "eastus2",
    "location": "eastus2",
    "time": "2021-02-21T15:39:46.172266Z",
    "componentName": "e